In [9]:
import torch
import torch.nn as nn
from model.pspnet import PSPNet
import cv2
import numpy as np

def check_data_types(model, input_tensor):
    def forward_hook(module, input, output):
        if len(input) > 0:
            input_type = input[0].dtype
        else:
            input_type = None
        output_type = output.dtype
        print(f"{module.__class__.__name__} - Input type: {input_type}, Output type: {output_type}")
    
    handles = []
    for layer in model.children():
        handle = layer.register_forward_hook(forward_hook)
        handles.append(handle)
    
    model(input_tensor)
    
    for handle in handles:
        handle.remove()

model = PSPNet(num_classes=4, dropout_p=0.5, use_aux=True)
input_a = cv2.imread("/mnt/e/VScode/WS-Hub/WS-U2net/U-2-Net/SEM_DATA/chged_images_256_a50_c80/Image1 - 001_0_0.jpg")
input_b = cv2.imread("/mnt/e/VScode/WS-Hub/WS-U2net/U-2-Net/SEM_DATA/chged_images_256_a50_c80/Image1 - 001_0_4.jpg")
input_n = np.stack((input_a, input_b), axis=0)
input_tensor = torch.tensor(input_n)
input_tensor = input_tensor.permute(0, 3, 1, 2).float()

check_data_types(model, input_tensor)

Sequential - Input type: torch.float32, Output type: torch.float32
Sequential - Input type: torch.float32, Output type: torch.float32
Dropout2d - Input type: torch.float32, Output type: torch.float32
Sequential - Input type: torch.float32, Output type: torch.float32
Dropout2d - Input type: torch.float32, Output type: torch.float32
Sequential - Input type: torch.float32, Output type: torch.float32
Dropout2d - Input type: torch.float32, Output type: torch.float32
Sequential - Input type: torch.float32, Output type: torch.float32
Dropout2d - Input type: torch.float32, Output type: torch.float32
PSPModule - Input type: torch.float32, Output type: torch.float32
Sequential - Input type: torch.float32, Output type: torch.float32
Sequential - Input type: torch.float32, Output type: torch.float32
